In [54]:
import numpy as np
import scipy 
from scipy import stats
from scipy.stats import norm,uniform
import matplotlib.pyplot as plt

In [92]:
Power = np.array([35, 38.26, 39.39, 40.56, 43.23, 56.54, 72.70, 84.19, 76.06, 61.32, 50.75, 46.90])
Gas = np.array([3, 2.81, 2.76, 2.73, 3.20, 3.88, 4.45, 4.8, 4.56, 4.13, 3.68, 3.49])
PowerTheta = np.array([38.29018572, 37.65306039, 38.00855419, 38.73256273, 39.36926748,
       39.86405353, 40.22086124, 40.44589774, 40.54870484, 40.55045989,
       40.70095953, 41.46173063, 43.25248543, 45.98926726, 49.29566467,
       52.83556305, 56.52346982, 60.36269157, 64.3524202 , 68.47769359,
       72.71945054, 76.89905869, 80.51516399, 83.03401433, 84.15904199,
       83.89240735, 82.26844207, 79.52729184, 76.07251473, 72.2958952 ,
       68.45430718, 64.73791746, 61.30609504, 58.17345317, 55.31154583,
       52.7660452 , 50.79182992, 49.67498941, 49.26046062, 48.6334227 ,
       46.83642493, 43.77355919, 40.27918283, 37.20522713, 35.04525891,
       34.04924416, 34.40661727, 35.92368012])
GasTheta = np.array([2.81145084, 2.76304781, 2.75516369, 2.76012114, 2.75811   ,
       2.74807589, 2.73185041, 2.71914404, 2.7312581 , 2.78949875,
       2.89661167, 3.03815321, 3.1995049 , 3.37083627, 3.54509695,
       3.71581731, 3.87964756, 4.03434353, 4.17888279, 4.31633758,
       4.45059611, 4.57966876, 4.68967835, 4.76550224, 4.79886395,
       4.79010961, 4.74061195, 4.65922203, 4.56072849, 4.45868332,
       4.35503145, 4.24597864, 4.12896081, 4.00748103, 3.88685281,
       3.77417548, 3.68158763, 3.6219074 , 3.59146408, 3.55818958,
       3.4882443 , 3.37526588, 3.24256103, 3.11387685, 3.0000309 ,
       2.9030518 , 2.82676425, 2.78845998])

In [93]:
PowerTheta.shape
Power.shape

(12,)

In [108]:
def meanReversion(random_seed , 
                  PowerPrices,
                  GasPrices,
                  PowerTheta,
                  GasTheta,
                  rho = 0.3,
                  T = 2):
    
    
    # N: number of intervals
    # n: number of paths
    # T: maturity
    # dt: interval of each path
    
    n = 100
    np.random.seed(random_seed)
    dt = 1/48 
    N = int(np.floor(T/dt))
    
    #params
    alphaCC = 7
    alphaJC = 20
    alphaG = 3
    m = 75
    p = 0.083# lambda*dt, weekly probability
    CC0 = 35
    JC0 = 0 
    G0 = 3
    spike_thres = 75
    sigmaG = 7.22/np.sqrt(dt)
    
    #V summer and winter
    v_summer = 28.87/np.sqrt(dt)
    v_winter = 10.83/np.sqrt(dt)
    V = np.zeros((N+1,n))
    
    #
    W = norm.rvs(size = (N+1,n))*np.sqrt(dt)
    Wtilde = norm.rvs(size = (N+1,n))*np.sqrt(dt)
    B = rho*W + np.sqrt(1-rho**2)*Wtilde
    
    #
    CC = np.zeros((N+1,n)) 
    CC[0,:] = CC0
    
    JC = np.zeros((N+1,n))
    JC[0,:] = JC0
    
    G = np.zeros((N+1,n)) 
    G[0,:] = G0
    
    PC = np.zeros((N+1,n))
    PC[0,:] = CC[0,:]
    # 
    Power_MSE = 0
    Gas_MSE = 0
    
    
    for i in range(1,N+1):
        month = np.ceil(i/4)%12 
        month = int((month == 0)*12 + (month != 0)*month)
        monthIndicator = (month > 4)&(month < 9)
        
        V[i,:] = monthIndicator*v_summer + (1 - monthIndicator)*v_winter
        
        CC[i,:] += alphaCC*(PowerTheta[(i-1)%48] - CC[i-1,:])*dt + V[i,:]*CC[i-1,:]*W[i,:]
        JC[i,:] += alphaJC*( 0 - JC[i-1,:])*dt + m*(uniform.rvs() < p)
        #Power Price
        PC[i,:] = CC[i,:] + JC[i,:]*(PC[i-1,:]>spike_thres)
        #Gas Price
        G[i,:]  += alphaG*(GasTheta[(i-1)%48] - G[i-1,:])*dt + sigmaG*G[i-1,:]*B[i,:]
        if i%4 == 0:
            Power_MSE += np.abs(np.mean(PC[i,:]) - PowerPrices[month - 1])
            Gas_MSE += np.abs(np.mean(G[i,:]) - GasPrices[month - 1])
            
            
    return Power_MSE,Gas_MSE
meanReversion(12,Power,Gas,PowerTheta,GasTheta )

(1.8779172629472363e+97, 1.1385304931748542e+63)

In [80]:
N = 2*48

0

In [36]:
v_summer = 28.87/np.sqrt(dt)
v_winter = 10.83/np.sqrt(dt)
print(v_summer)
print(v_winter)

200.01722725805396
75.03244098388377
